In [2]:
def _read_stripes(file_name):
    
    hold_dict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}
    source_dict = {5:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]},3:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}}
    
    check_c = [f"chr{n}" for n in range(1,23)] + ["chrX"]
    var5 = []
    var3 = []
    incre=-1
#     with open("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/GM12878.bedpe", "r") as f:
    with open(file_name, "r") as f:
        f.readline()
        for line in f:
            chromo,c1,c2,_,c4,c5,pval=line.rstrip("\n").split()
            if chromo != check_c[0]:
                check_c.pop(0)
            incre+=1
            if int(c2)-int(c1)<int(c5)-int(c4):
                source_dict[5][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]+[float(pval)]))]
            else:
                source_dict[3][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]+[float(pval)]))]
                
    return source_dict, hold_dict

def _read_peaks(file_name):
    
    hold_pdict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]+["chrY"]}
    source_pdict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]+["chrY"]}
    
    check_c = [f"chr{n}" for n in range(1,23)] + ["chrX"] + ["chrY"]
    var5 = []
    var3 = []
    incre=-1
#     with open("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/GM12878.bedpe", "r") as f:
    with open(file_name, "r") as f:
        f.readline()
        for line in f:
            chromo,c1,c2,payload=line.split()
            if chromo != check_c[0]:
                check_c.pop(0)
            incre+=1
#             if int(c2)-int(c1)<int(c5)-int(c4):
            source_pdict[chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,payload]]))]
                
    return source_pdict, hold_pdict



def IntersectingIntervals(arr1, w_1, arr2, spacer=50000):
    i = j = 0
    n = len(arr1)
    m = len(arr2)
        
    result=[]
    while i < n and j < m:
        while (j < m and arr2[j][3]<arr1[i][w_1]):
            j+=1
            
        if j>=m: #one level up?
            break
        
        if arr1[i][w_1]-spacer<=arr2[j][2]<=arr1[i][w_1+1]+spacer and arr1[i][w_1]-spacer<=arr2[j][3]<=arr1[i][w_1+1]+spacer and arr2[j][-1]!=0:
            result+=[arr1[i]]
            i+=1
        elif arr2[j][2]>arr1[i][w_1+1]+spacer:
            i+=1
        else:
            j+=1
        
    return result


def _load_Intersect(source_dict,source_peaks,hold_dict,spacer):
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        hold_dict[key]=IntersectingIntervals(source_dict[5][key],2,source_peaks[key],spacer=2000)
        hold_dict[key]+=IntersectingIntervals(source_dict[3][key],4,source_peaks[key],spacer=2000)
        
def _write_Intersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            payload = value[-1]
            c5=value[-2]
            c4=value[-3]
            write_out+="\t".join([str(y) for y in value[1:4]]+[value[1]]+[str(c4)]+[str(c5)]+[str(payload)])+"\n"
            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)
        
        
def _z_write_Intersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            svalue = [str(x) for x in value]

            write_out+="\t".join([y for y in svalue[1:4]]+[svalue[1]]+[z for z in svalue[4:]])+"\n"
            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)
        
def _snn_write_Intersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            svalue = [str(x) for x in value]
            write_out+="\t".join([y for y in svalue[1:4]]+[svalue[1]]+[z for z in svalue[4:]])+"\n"            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)
        
def call_CTCF_stripes(stripename, ctcfname, fileoutname,spacer=50000):
    sdict, hdict=_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_Intersect(sdict, source_pdict,hdict,spacer)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _write_Intersect(hdict, fileoutname)
    
def z_call_CTCF_stripes(stripename, ctcfname, fileoutname,spacer=50000):
    sdict, hdict=_zebra_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_Intersect(sdict, source_pdict,hdict,spacer)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _z_write_Intersect(hdict, fileoutname)
    
def snn_call_CTCF_stripes(stripename, ctcfname, fileoutname,spacer=50000):
    sdict, hdict=_stripeNN_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_Intersect(sdict, source_pdict,hdict,spacer)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _snn_write_Intersect(hdict, fileoutname)
    
    
def noIntersectingIntervals(arr1, w_1, arr2, spacer=50000):
    i = j = 0
    n = len(arr1)
    m = len(arr2)
        
    result=[]
    while i < n and j < m:
        while (j < m and arr2[j][3]<arr1[i][w_1]):
            j+=1
            
        if j>=m: #one level up?
            break
        
        if arr1[i][w_1]-spacer<=arr2[j][2]<=arr1[i][w_1+1]+spacer and arr1[i][w_1]-spacer<=arr2[j][3]<=arr1[i][w_1+1]+spacer and arr2[j][-1]!=0:
            i+=1
        elif arr2[j][2]>arr1[i][w_1+1]+spacer:
            result+=[arr1[i]]
            i+=1
        else:
            j+=1
        
    return result

def _load_noIntersect(source_dict,source_peaks,hold_dict,spacer=2000):
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        hold_dict[key]=noIntersectingIntervals(source_dict[5][key],2,source_peaks[key],spacer)
        hold_dict[key]+=noIntersectingIntervals(source_dict[3][key],4,source_peaks[key],spacer)
        
def _write_noIntersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            payload = value[-1]
            c5=value[-2]
            c4=value[-3]
            write_out+="\t".join([str(y) for y in value[1:4]]+[value[1]]+[str(c4)]+[str(c5)]+[str(payload)])+"\n"
            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)

def _z_write_noIntersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            svalue = [str(x) for x in value]

            write_out+="\t".join([y for y in svalue[1:4]]+[svalue[1]]+[z for z in svalue[4:]])+"\n"
            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)

def _snn_write_noIntersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            svalue = [str(x) for x in value]
            write_out+="\t".join([y for y in svalue[1:4]]+[svalue[1]]+[z for z in svalue[4:]])+"\n"            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)


def call_non_CTCF_stripes(stripename, ctcfname, fileoutname,spacer=1000):
    sdict, hdict=_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_noIntersect(sdict, source_pdict,hdict,spacer)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _write_noIntersect(hdict, fileoutname)

def z_call_non_CTCF_stripes(stripename, ctcfname, fileoutname,spacer=1000):
    sdict, hdict=_zebra_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_noIntersect(sdict, source_pdict,hdict,spacer)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _z_write_noIntersect(hdict, fileoutname)
    
def snn_call_non_CTCF_stripes(stripename, ctcfname, fileoutname,spacer=1000):
    sdict, hdict=_stripeNN_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_noIntersect(sdict, source_pdict,hdict,spacer)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _snn_write_noIntersect(hdict, fileoutname)

In [3]:
def read_intervals_by_chromosome(file_path):
    """
    Reads intervals from File A and organizes them in a dictionary keyed by chromosome.
    """
    chrom_intervals = {}
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 3:
                chrom = parts[0]
                interval = (parts[1], parts[2])  # (start, end) as strings
                if chrom not in chrom_intervals:
                    chrom_intervals[chrom] = set()
                chrom_intervals[chrom].add(interval)
    return chrom_intervals

def find_matching_loops_by_chromosome(file_a_path, file_b_path, output_path=None):
    """
    Finds and prints (or saves) loops from File B that match any interval from File A.
    Matching is chromosome-aware.
    """
    intervals_by_chr = read_intervals_by_chromosome(file_a_path)

    with open(file_b_path, 'r') as file_b:
        output_lines = []
        for line in file_b:
            parts = line.strip().split()
            if len(parts) >= 6:
                chrom1, start1, end1 = parts[0:3]
                chrom2, start2, end2 = parts[3:6]
                
                match1 = chrom1 in intervals_by_chr and (start1, end1) in intervals_by_chr[chrom1]
                match2 = chrom2 in intervals_by_chr and (start2, end2) in intervals_by_chr[chrom2]
                
                if match1 or match2:
                    output_lines.append(line)

    # Output results
    if output_path:
        with open(output_path, 'w') as out_f:
            out_f.writelines(output_lines)
    else:
        for line in output_lines:
            print(line.strip())


In [ ]:
%%bash
prefix="./data_files/10KQuaggaCalls/"
sourcefile="10000nt_MR10000000_ML50000_MD2000000_MW80000_WS2_SG1RH0.1_NSTR2_WholeGenome_hg38_GM12878_width_filtered.txt"
zebra='../stripe_results/GM12878.bedpe'
stripenn='../stripe_results/results_filtered.tsv'

awk '$3 - $2 <= $6 - $5 { OFS = "\t"; print $1, $2, $3, $4, $5, $6, $7 }' ${prefix}${sourcefile} > ./uniqueAnalysis/v_or_h/q_5prime.txt
awk '$3 - $2 > $6 - $5 { OFS = "\t"; print $1, $2, $3, $4, $5, $6, $7 }' ${prefix}${sourcefile} > ./uniqueAnalysis/v_or_h/q_3prime.txt

awk '$3 - $2 <= $6 - $5 { OFS = "\t"; print $1, $2, $3, $4, $5, $6, $7 }' ${zebra} > ./uniqueAnalysis/v_or_h/z_5prime.txt
awk '$3 - $2 > $6 - $5 { OFS = "\t"; print $1, $2, $3, $4, $5, $6, $7 }' ${zebra} > ./uniqueAnalysis/v_or_h/z_3prime.txt

# stripenn has different
awk -F'\t' '{ if (int($2) == int($5)) print $1, $2, $3, $4, $5, $6, $7 }' ${stripenn} > ./uniqueAnalysis/v_or_h/s_5prime.txt
awk -F'\t' '{ if (int($2) != int($5)) print $1, $2, $3, $4, $5, $6, $7 }' ${stripenn} > ./uniqueAnalysis/v_or_h/s_3prime.txt

awk '{ OFS = "\t"; print $1, $2, $3 }' ./uniqueAnalysis/v_or_h/q_5prime.txt > ./uniqueAnalysis/v_or_h/q_5prime_cut.txt
awk '{ OFS = "\t"; print $1, $2, $3 }' ./uniqueAnalysis/v_or_h/z_5prime.txt > ./uniqueAnalysis/v_or_h/z_5prime_cut.txt
awk '{ OFS = "\t"; print $1, $2, $3 }' ./uniqueAnalysis/v_or_h/s_5prime.txt > ./uniqueAnalysis/v_or_h/s_5prime_cut.txt

awk '{ OFS = "\t"; print $4, $5, $6 }' ./uniqueAnalysis/v_or_h/q_3prime.txt > ./uniqueAnalysis/v_or_h/q_3prime_cut.txt
awk '{ OFS = "\t"; print $4, $5, $6 }' ./uniqueAnalysis/v_or_h/z_3prime.txt > ./uniqueAnalysis/v_or_h/z_3prime_cut.txt
awk '{ OFS = "\t"; print $4, $5, $6 }' ./uniqueAnalysis/v_or_h/s_3prime.txt > ./uniqueAnalysis/v_or_h/s_3prime_cut.txt


In [ ]:
%%bash
# use uniq, as many can fit within larger stripes.
mkdir unqiueAnalysis
mkdir ./unqiueAnalysis/v_or_h
mkdir ./unqiueAnalysis/v_or_h/unique
mkdir ./unqiueAnalysis/finalout

bedtools intersect -v -a ./unqiueAnalysis/v_or_h/q_5prime_cut.txt -b ./unqiueAnalysis/v_or_h/z_5prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/q_no_z_5prime_cut.txt
bedtools intersect -v -a ./unqiueAnalysis/v_or_h/q_3prime_cut.txt -b ./unqiueAnalysis/v_or_h/z_3prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/q_no_z_3prime_cut.txt

bedtools intersect -v -a ./unqiueAnalysis/v_or_h/q_5prime_cut.txt -b ./unqiueAnalysis/v_or_h/s_5prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/q_no_s_5prime_cut.txt
bedtools intersect -v -a ./unqiueAnalysis/v_or_h/q_3prime_cut.txt -b ./unqiueAnalysis/v_or_h/s_3prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/q_no_s_3prime_cut.txt



bedtools intersect -v -a ./unqiueAnalysis/v_or_h/z_5prime_cut.txt -b ./unqiueAnalysis/v_or_h/q_5prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/z_no_q_5prime_cut.txt
bedtools intersect -v -a ./unqiueAnalysis/v_or_h/z_3prime_cut.txt -b ./unqiueAnalysis/v_or_h/q_3prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/z_no_q_3prime_cut.txt

bedtools intersect -v -a ./unqiueAnalysis/v_or_h/z_5prime_cut.txt -b ./unqiueAnalysis/v_or_h/s_5prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/z_no_s_5prime_cut.txt
bedtools intersect -v -a ./unqiueAnalysis/v_or_h/z_3prime_cut.txt -b ./unqiueAnalysis/v_or_h/s_3prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/z_no_s_3prime_cut.txt



bedtools intersect -v -a ./unqiueAnalysis/v_or_h/s_5prime_cut.txt -b ./unqiueAnalysis/v_or_h/q_5prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/s_no_q_5prime_cut.txt
bedtools intersect -v -a ./unqiueAnalysis/v_or_h/s_3prime_cut.txt -b ./unqiueAnalysis/v_or_h/q_3prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/s_no_q_3prime_cut.txt

bedtools intersect -v -a ./unqiueAnalysis/v_or_h/s_5prime_cut.txt -b ./unqiueAnalysis/v_or_h/z_5prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/s_no_z_5prime_cut.txt
bedtools intersect -v -a ./unqiueAnalysis/v_or_h/s_3prime_cut.txt -b ./unqiueAnalysis/v_or_h/z_3prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/unique/s_no_z_3prime_cut.txt



# Intersection between all tools, perspective of q_*, s_*, or z_*

bedtools intersect -v -a ./unqiueAnalysis/v_or_h/intersects/q_no_z_5prime_cut.txt -b ./unqiueAnalysis/v_or_h/s_5prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/intersects/q_no_zs_5prime_cut.txt
bedtools intersect -v -a ./unqiueAnalysis/v_or_h/intersects/q_no_z_3prime_cut.txt -b ./unqiueAnalysis/v_or_h/s_3prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/intersects/q_no_zs_3prime_cut.txt

bedtools intersect -v -a ./unqiueAnalysis/v_or_h/intersects/s_no_z_5prime_cut.txt -b ./unqiueAnalysis/v_or_h/q_5prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/intersects/s_no_zq_5prime_cut.txt
bedtools intersect -v -a ./unqiueAnalysis/v_or_h/intersects/s_no_z_3prime_cut.txt -b ./unqiueAnalysis/v_or_h/q_3prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/intersects/s_no_zq_3prime_cut.txt

bedtools intersect -v -a ./unqiueAnalysis/v_or_h/intersects/z_no_q_5prime_cut.txt -b ./unqiueAnalysis/v_or_h/s_5prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/intersects/z_no_qs_5prime_cut.txt
bedtools intersect -v -a ./unqiueAnalysis/v_or_h/intersects/z_no_q_3prime_cut.txt -b ./unqiueAnalysis/v_or_h/s_3prime_cut.txt| uniq  > ./unqiueAnalysis/v_or_h/intersects/z_no_qs_3prime_cut.txt


In [ ]:
# %%bash

# #may need to remove headers from final files below

# cat ./unqiueAnalysis/v_or_h/intersects/q_no_zs_5prime_cut.txt ./unqiueAnalysis/v_or_h/intersects/q_no_zs_3prime_cut.txt > ./unqiueAnalysis/finalout/q_no_zs_final_cut.txt
# cat ./unqiueAnalysis/v_or_h/intersects/s_no_zq_5prime_cut.txt ./unqiueAnalysis/v_or_h/intersects/s_no_zq_3prime_cut.txt > ./unqiueAnalysis/finalout/s_no_zq_final_cut.txt
# cat ./unqiueAnalysis/v_or_h/intersects/z_no_qs_5prime_cut.txt ./unqiueAnalysis/v_or_h/intersects/z_no_qs_3prime_cut.txt > ./unqiueAnalysis/finalout/z_no_qs_final_cut.txt

In [ ]:
!mkdir ./unqiueAnalysis/remapped

In [6]:
stripepath={
    'Quagga': "./unqiueAnalysis/finalout/q_no_zs_final_cut.txt",
    'Zebra': "./unqiueAnalysis/finalout/s_no_zq_final_cut.txt",
    'StripeNN': "./unqiueAnalysis/finalout/z_no_qs_final_cut.txt"
}

pathA = "./unqiueAnalysis/v_or_h/q_3prime.txt"
pathB = "./unqiueAnalysis/v_or_h/z_3prime.txt"
pathC = "./unqiueAnalysis/v_or_h/s_3prime.txt"

find_matching_loops_by_chromosome(stripepath['Quagga'], pathA, "./unqiueAnalysis/remapped/"+"quagga_uniq_3prime_remapped.txt")
find_matching_loops_by_chromosome(stripepath['Zebra'], pathB, "./unqiueAnalysis/remapped/"+"zebra_uniq_3prime_remapped.txt")
find_matching_loops_by_chromosome(stripepath['StripeNN'], pathC, "./unqiueAnalysis/remapped/"+"stripenn_uniq_3prime_remapped.txt")

In [ ]:
%%bash

###!!!! fuse 5 and 3remapped and make into new epigenomic pileups and APA.
# may need to remove headers manually "#chr x1 x2, or chr pos1 pos2, etc."

cat ./unqiueAnalysis/v_or_h/q_5prime.txt ./unqiueAnalysis/remapped/quagga_uniq_3prime_remapped.txt > ./unqiueAnalysis/finalout/q_no_zs_final_cut.txt
cat ./unqiueAnalysis/v_or_h/s_5prime.txt ./unqiueAnalysis/remapped/stripenn_uniq_3prime_remapped.txt > ./unqiueAnalysis/finalout/s_no_zq_final_cut.txt
cat ./unqiueAnalysis/v_or_h/z_5prime.txt ./unqiueAnalysis/remapped/zebra_uniq_3prime_remapped.txt > ./unqiueAnalysis/finalout/z_no_qs_final_cut.txt